<table class="tfo-notebook-buttons" align="left">
  <td>
    <a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/nlp/c4_w1_01_stack_semantics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>    
  </td>
  <td>
    <a href="https://github.com/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/nlp/c4_w1_01_stack_semantics.ipynb" target="_parent"><img src="https://raw.githubusercontent.com/martin-fabbri/colab-notebooks/master/assets/github.svg" alt="View On Github"/></a>  </td>
</table>

In [1]:
%env PYTHONPATH=

env: PYTHONPATH=


In [2]:
%%capture
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-latest-Linux-x86_64.sh

MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

conda install --channel defaults conda python=3.8 --yes
conda update --channel defaults --all --yes

In [3]:
!conda --version

conda 4.9.2


In [4]:
!python --version

Python 3.8.5


In [5]:
%%capture
%%bash
pip install streamlit
pip install st-annotated-text

In [6]:
# import os
# os.kill(os.getpid(), 9)

In [7]:
%%capture
%%bash
npm install -g npm
npm install localtunnel

In [9]:
%%writefile requirements.txt
requests==2.25.1
tensorflow==2.4.1
streamlit==0.76.0
google_api_python_client==1.12.8
protobuf==3.14.0

Writing requirements.txt


In [10]:
#%%capture
%%bash
pip install -r requirements.txt 

In [25]:
%%writefile SessionState.py
"""Hack to add per-session state to Streamlit.
Usage
-----
>>> import SessionState
>>>
>>> session_state = SessionState.get(user_name='', favorite_color='black')
>>> session_state.user_name
''
>>> session_state.user_name = 'Mary'
>>> session_state.favorite_color
'black'
Since you set user_name above, next time your script runs this will be the
result:
>>> session_state = get(user_name='', favorite_color='black')
>>> session_state.user_name
'Mary'
"""
try:
    import streamlit.ReportThread as ReportThread
    from streamlit.server.Server import Server
except Exception:
    # Streamlit >= 0.65.0
    import streamlit.report_thread as ReportThread
    from streamlit.server.server import Server


class SessionState(object):
    def __init__(self, **kwargs):
        """A new SessionState object.
        Parameters
        ----------
        **kwargs : any
            Default values for the session state.
        Example
        -------
        >>> session_state = SessionState(user_name='', favorite_color='black')
        >>> session_state.user_name = 'Mary'
        ''
        >>> session_state.favorite_color
        'black'
        """
        for key, val in kwargs.items():
            setattr(self, key, val)


def get(**kwargs):
    """Gets a SessionState object for the current session.
    Creates a new object if necessary.
    Parameters
    ----------
    **kwargs : any
        Default values you want to add to the session state, if we're creating a
        new one.
    Example
    -------
    >>> session_state = get(user_name='', favorite_color='black')
    >>> session_state.user_name
    ''
    >>> session_state.user_name = 'Mary'
    >>> session_state.favorite_color
    'black'
    Since you set user_name above, next time your script runs this will be the
    result:
    >>> session_state = get(user_name='', favorite_color='black')
    >>> session_state.user_name
    'Mary'
    """
    # Hack to get the session object from Streamlit.

    ctx = ReportThread.get_report_ctx()

    this_session = None

    current_server = Server.get_current()
    if hasattr(current_server, "_session_infos"):
        # Streamlit < 0.56
        session_infos = Server.get_current()._session_infos.values()
    else:
        session_infos = Server.get_current()._session_info_by_id.values()
    for session_info in session_infos:
        s = session_info.session
        if (
            # Streamlit < 0.54.0
            (hasattr(s, "_main_dg") and s._main_dg == ctx.main_dg)
            or
            # Streamlit >= 0.54.0
            (not hasattr(s, "_main_dg") and s.enqueue == ctx.enqueue)
            or
            # Streamlit >= 0.65.2
            (
                not hasattr(s, "_main_dg")
                and s._uploaded_file_mgr == ctx.uploaded_file_mgr
            )
        ):
            this_session = s
    if this_session is None:
        raise RuntimeError(
            "Oh noes. Couldn't get your Streamlit Session object. "
            "Are you doing something fancy with threads?"
        )
    # Got the session object! Now let's attach some state into it.

    if not hasattr(this_session, "_custom_session_state"):
        this_session._custom_session_state = SessionState(**kwargs)
    return this_session._custom_session_state

Overwriting SessionState.py


In [26]:
%%writefile utils.py
# Utils for preprocessing data etc
import tensorflow as tf
import googleapiclient.discovery
from google.api_core.client_options import ClientOptions

base_classes = [
    "chicken_curry",
    "chicken_wings",
    "fried_rice",
    "grilled_salmon",
    "hamburger",
    "ice_cream",
    "pizza",
    "ramen",
    "steak",
    "sushi",
]

classes_and_models = {
    "model_1": {
        "classes": base_classes,
        "model_name": "vision_food_delete_me",  # change to be your model name
    },
    "model_2": {
        "classes": sorted(base_classes + ["donut"]),
        "model_name": "efficientnet_model_2_11_classes",
    },
    "model_3": {
        "classes": sorted(base_classes + ["donut", "not_food"]),
        "model_name": "efficientnet_model_3_12_classes",
    },
}


def predict_json(project, region, model, instances, version=None):
    """Send json data to a deployed model for prediction.
    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to Tensors.
        version (str): version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    # Create the ML Engine service object
    prefix = "{}-ml".format(region) if region else "ml"
    api_endpoint = "https://{}.googleapis.com".format(prefix)
    client_options = ClientOptions(api_endpoint=api_endpoint)

    # Setup model path
    model_path = "projects/{}/models/{}".format(project, model)
    if version is not None:
        model_path += "/versions/{}".format(version)
    # Create ML engine resource endpoint and input data
    ml_resource = googleapiclient.discovery.build(
        "ml", "v1", cache_discovery=False, client_options=client_options
    ).projects()
    instances_list = (
        instances.numpy().tolist()
    )  # turn input into list (ML Engine wants JSON)

    input_data_json = {
        "signature_name": "serving_default",
        "instances": instances_list,
    }

    request = ml_resource.predict(name=model_path, body=input_data_json)
    response = request.execute()

    # # ALT: Create model api
    # model_api = api_endpoint + model_path + ":predict"
    # headers = {"Authorization": "Bearer " + token}
    # response = requests.post(model_api, json=input_data_json, headers=headers)

    if "error" in response:
        raise RuntimeError(response["error"])
    return response["predictions"]


# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224, rescale=False):
    """
    Reads in an image from filename, turns it into a tensor and reshapes into
    (224, 224, 3).
    """
    # Decode it into a tensor
    #   img = tf.io.decode_image(filename) # no channels=3 means model will break for some PNG's (4 channels)
    img = tf.io.decode_image(
        filename, channels=3
    )  # make sure there's 3 colour channels (for PNG's)
    # Resize the image
    img = tf.image.resize(img, [img_shape, img_shape])
    # Rescale the image (get all values between 0 and 1)
    if rescale:
        return img / 255.0
    else:
        return img


def update_logger(
    image, model_used, pred_class, pred_conf, correct=False, user_label=None
):
    """
    Function for tracking feedback given in app, updates and reutrns
    logger dictionary.
    """
    logger = {
        "image": image,
        "model_used": model_used,
        "pred_class": pred_class,
        "pred_conf": pred_conf,
        "correct": correct,
        "user_label": user_label,
    }
    return logger

Overwriting utils.py


In [27]:
%%writefile app.py
import os
import json
import requests
import SessionState
import streamlit as st
import tensorflow as tf
from utils import (
    load_and_prep_image,
    classes_and_models,
    update_logger,
    predict_json,
)

# Setup environment credentials (you'll need to change these)
os.environ[
    "GOOGLE_APPLICATION_CREDENTIALS"
] = "/content/deeplearning-300314-28fd9a664766.json"  # change for your GCP key
PROJECT = "deeplearning-300314"  # change for your GCP project

# https://gcping.com/
# Oregon, USA
# us-west1
REGION = (
    "us-west1"
)

### Streamlit code (works as a straigtht-forward script) ###
st.title("Welcome to Food Vision 🍔📸")
st.header("Identify what's in your food photos!")


@st.cache  # cache the function so predictions aren't always redone (Streamlit refreshes every click)
def make_prediction(image, model, class_names):
    """
    Takes an image and uses model (a trained TensorFlow model) to make a
    prediction.

    Returns:
     image (preproccessed)
     pred_class (prediction class from class_names)
     pred_conf (model confidence)
    """
    image = load_and_prep_image(image)
    # Turn tensors into int16 (saves a lot of space, ML Engine has a limit of 1.5MB per request)
    image = tf.cast(tf.expand_dims(image, axis=0), tf.int16)
    # image = tf.expand_dims(image, axis=0)
    preds = predict_json(
        project=PROJECT, region=REGION, model=model, instances=image
    )
    pred_class = class_names[tf.argmax(preds[0])]
    pred_conf = tf.reduce_max(preds[0])
    return image, pred_class, pred_conf


# Pick the model version
choose_model = st.sidebar.selectbox(
    "Pick model you'd like to use",
    (
        "Model 1 (10 food classes)",  # original 10 classes
        "Model 2 (11 food classes)",  # original 10 classes + donuts
        "Model 3 (11 food classes + non-food class)",
    ),  # 11 classes (same as above) + not_food class
)

# Model choice logic
if choose_model == "Model 1 (10 food classes)":
    CLASSES = classes_and_models["model_1"]["classes"]
    MODEL = classes_and_models["model_1"]["model_name"]
elif choose_model == "Model 2 (11 food classes)":
    CLASSES = classes_and_models["model_2"]["classes"]
    MODEL = classes_and_models["model_2"]["model_name"]
else:
    CLASSES = classes_and_models["model_3"]["classes"]
    MODEL = classes_and_models["model_3"]["model_name"]
# Display info about model and classes
if st.checkbox("Show classes"):
    st.write(
        f"You chose {MODEL}, these are the classes of food it can identify:\n",
        CLASSES,
    )
# File uploader allows user to add their own image
uploaded_file = st.file_uploader(
    label="Upload an image of food", type=["png", "jpeg", "jpg"]
)

# Setup session state to remember state of app so refresh isn't always needed
# See: https://discuss.streamlit.io/t/the-button-inside-a-button-seems-to-reset-the-whole-app-why/1051/11
session_state = SessionState.get(pred_button=False)

# Create logic for app flow
if not uploaded_file:
    st.warning("Please upload an image.")
    st.stop()
else:
    session_state.uploaded_image = uploaded_file.read()
    st.image(session_state.uploaded_image, use_column_width=True)
    pred_button = st.button("Predict")
# Did the user press the predict button?
if pred_button:
    session_state.pred_button = True
# And if they did...
if session_state.pred_button:
    (
        session_state.image,
        session_state.pred_class,
        session_state.pred_conf,
    ) = make_prediction(
        session_state.uploaded_image, model=MODEL, class_names=CLASSES
    )
    st.write(
        f"Prediction: {session_state.pred_class}, \
               Confidence: {session_state.pred_conf:.3f}"
    )

    # Create feedback mechanism (building a data flywheel)
    session_state.feedback = st.selectbox(
        "Is this correct?", ("Select an option", "Yes", "No")
    )
    if session_state.feedback == "Select an option":
        pass
    elif session_state.feedback == "Yes":
        st.write("Thank you for your feedback!")
        # Log prediction information to terminal (this could be stored in Big Query or something...)
        print(
            update_logger(
                image=session_state.image,
                model_used=MODEL,
                pred_class=session_state.pred_class,
                pred_conf=session_state.pred_conf,
                correct=True,
            )
        )
    elif session_state.feedback == "No":
        session_state.correct_class = st.text_input(
            "What should the correct label be?"
        )
        if session_state.correct_class:
            st.write(
                "Thank you for that, we'll use your help to make our model better!"
            )
            # Log prediction information to terminal (this could be stored in Big Query or something...)
            print(
                update_logger(
                    image=session_state.image,
                    model_used=MODEL,
                    pred_class=session_state.pred_class,
                    pred_conf=session_state.pred_conf,
                    correct=False,
                    user_label=session_state.correct_class,
                )
            )

Overwriting app.py


In [28]:
!streamlit run app.py &>/dev/null&

In [29]:
!curl http://localhost:8501

<!doctype html><html lang="en"><head><meta charset="UTF-8"/><meta name="viewport" content="width=device-width,initial-scale=1,shrink-to-fit=no"/><link rel="shortcut icon" href="./favicon.png"/><title>Streamlit</title><script src="./vendor/viz/viz-1.8.0.min.js" type="javascript/worker"></script><link href="./static/css/9.cbc425bc.chunk.css" rel="stylesheet"><link href="./static/css/main.5905f91f.chunk.css" rel="stylesheet"></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div><script>!function(e){function t(t){for(var c,n,d=t[0],a=t[1],u=t[2],i=0,s=[];i<d.length;i++)n=d[i],Object.prototype.hasOwnProperty.call(f,n)&&f[n]&&s.push(f[n][0]),f[n]=0;for(c in a)Object.prototype.hasOwnProperty.call(a,c)&&(e[c]=a[c]);for(l&&l(t);s.length;)s.shift()();return o.push.apply(o,u||[]),r()}function r(){for(var e,t=0;t<o.length;t++){for(var r=o[t],c=!0,n=1;n<r.length;n++){var a=r[n];0!==f[a]&&(c=!1)}c&&(o.splice(t--,1),e=d(d.s=r[0]))}return e}var c={},n={7:

In [30]:
!npx localtunnel --port 8501

your url is: https://dry-firefox-35.loca.lt
^C


In [32]:
%%writefile app.yaml
runtime: custom
env: flex

Overwriting app.yaml


In [34]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=w5RSU54PxwPzjF9e5V4akRZwr93zHz&prompt=consent&access_type=offline&code_challenge=N3Yzz4InNl98dp78kWHqcxn5Ec8uzcaduDSH3pttcmU&code_challenge_method=S256

Enter verification code: 4/1AY0e-g4mZCth3OyUHvPFyDx4Bl4QHAqVBJ7WTQSb44E467LbHRqMVWt1x6o

You are now logged in as [martin.fabbri.a@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [35]:
!gcloud config set project deeplearning-300314

Updated property [core/project].


In [37]:
%%writefile Dockerfile
FROM python:3.7

## App engine stuff
# Expose port you want your app on
EXPOSE 8080

# Upgrade pip 
RUN pip install -U pip

COPY requirements.txt app/requirements.txt
RUN pip install -r app/requirements.txt

# Create a new directory for app (keep it in its own directory)
COPY . /app
WORKDIR app

# Run
ENTRYPOINT ["streamlit", "run", "app.py", "--server.port=8080", "--server.address=0.0.0.0"] 

Writing Dockerfile


In [42]:
%%writefile .dockerignore
# *.json
*.jpg
*.jpeg
*.git
*.key
env
images
keynote-images

# Python stuff
*.pyc
*.pyo
*.pyd
__pycache__
.pytest_cache

Overwriting .dockerignore


In [43]:
!gcloud app deploy app.yaml

Services to deploy:

descriptor:      [/content/app.yaml]
source:          [/content]
target project:  [deeplearning-300314]
target service:  [default]
target version:  [20210215t103859]
target url:      [https://deeplearning-300314.wl.r.appspot.com]


Do you want to continue (Y/n)?  y

Beginning deployment of service [default]...
Building and pushing image for service [default]
Started cloud build [27d9e389-a9fb-48a2-887f-235ca6c7d54c].
To see logs in the Cloud Console: https://console.cloud.google.com/cloud-build/builds/27d9e389-a9fb-48a2-887f-235ca6c7d54c?project=959316098634
 REMOTE BUILD OUTPUT 
starting build "27d9e389-a9fb-48a2-887f-235ca6c7d54c"

FETCHSOURCE
Fetching storage object: gs://staging.deeplearning-300314.appspot.com/us.gcr.io/deeplearning-300314/appengine/default.20210215t103859:latest#1613385561160968
Copying gs://staging.deeplearning-300314.appspot.com/us.gcr.io/deeplearning-300314/appengine/default.20210215t103859:latest#1613385561160968...
\
Operation completed o

In [40]:
!gcloud app logs tail -s default

Waiting for new log entries...


Command killed by keyboard interrupt

^C
